In [ ]:
# # Cell 1

# !pip install --quiet requests beautifulsoup4 dateparser pandas tqdm pytz

In [ ]:
# Cell 2

import re
import time
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import dateparser
from datetime import datetime
import pytz
import pandas as pd


# ---------------------------
# CONFIG
# ---------------------------
CHANNEL = "inet"

BASE_DOMAIN = "https://inet.detik.com"
BASE_INDEX_URL = BASE_DOMAIN + "/indeks"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; scraping-bot/1.0)"
}

DELAY_SEC = 0.6
MAX_PAGES = 5
TIMEZONE = "Asia/Jakarta"
# ---------------------------

tz = pytz.timezone(TIMEZONE)

In [ ]:
# Cell 3

def absolutize(href, base_domain):
    if not href:
        return None
    if href.startswith("http"):
        return href
    return urljoin(base_domain, href)

def fetch(url, headers=HEADERS, max_retries=3, timeout=15):
    for attempt in range(1, max_retries + 1):
        try:
            r = requests.get(url, headers=headers, timeout=timeout)
            r.raise_for_status()
            return r.text
        except Exception as e:
            print(f"[fetch] gagal {url} (attempt {attempt}): {e}")
            time.sleep(0.4 + attempt * 0.4)
    return None


In [ ]:
# Cell 4

def extract_article_links(index_html, base_domain):
    soup = BeautifulSoup(index_html, "html.parser")
    links = set()

    for a in soup.find_all("a", href=True):
        href = a["href"].strip()

        # ❌ SKIP video / 20.detik.com
        if "20.detik.com" in href:
            continue

        if re.search(r"/d-\d+", href):
            abs_url = absolutize(href, base_domain)
            if abs_url and "20.detik.com" not in abs_url:
                links.add(abs_url)

    return sorted(links)

In [ ]:
# Cell 5

def parse_article(article_html, url=None):
    soup = BeautifulSoup(article_html, "html.parser")

    # Judul
    h1 = soup.find("h1")
    title = h1.get_text(strip=True) if h1 else None

    # Tanggal
    time_tag = soup.find("time")
    date_raw = time_tag.get_text(strip=True) if time_tag else None

    parsed_dt = None
    if date_raw:
        parsed_dt = dateparser.parse(
            date_raw,
            languages=["id"],
            settings={
                "TIMEZONE": TIMEZONE,
                "RETURN_AS_TIMEZONE_AWARE": True
            }
        )

    # Lead
    lead = None
    for cont in [
        soup.find("div", class_="detail__body-text"),
        soup.find("div", class_="detail__body"),
        soup.find("article"),
    ]:
        if cont:
            p = cont.find("p")
            if p and p.get_text(strip=True):
                lead = p.get_text(" ", strip=True)
                break

    return {
        "url": url,
        "title": title,
        "date_raw": date_raw,
        "date_parsed": parsed_dt,
        "lead": lead
    }


In [ ]:
# Cell 6

results = []
seen_urls = set()

current_url = BASE_INDEX_URL

pbar = tqdm(total=MAX_PAGES, desc="pages inet", unit="page")

for page_no in range(1, MAX_PAGES + 1):
    index_url = f"{BASE_INDEX_URL}?page={page_no}"
    print(f"[inet] page {page_no} -> {index_url}")

    idx_html = fetch(index_url)
    if not idx_html:
        print("[inet] gagal ambil indeks → STOP")
        break

    article_links = extract_article_links(idx_html, BASE_DOMAIN)
    print(f"  ditemukan {len(article_links)} artikel")

    if not article_links:
        print("[inet] tidak ada artikel → STOP")
        break

    for link in article_links:
        if link in seen_urls:
            continue
        seen_urls.add(link)

        time.sleep(DELAY_SEC)

        art_html = fetch(link)
        if not art_html:
            continue

        info = parse_article(art_html, url=link)

        if not info["title"] or not info["date_parsed"]:
            continue

        results.append({
            "channel": CHANNEL,
            "url": link,
            "judul": info["title"],
            "tanggal": info["date_raw"],
            "tanggal_iso": info["date_parsed"].isoformat(),
            "lead": info["lead"]
        })

        print(f"   + {info['title']}")

    pbar.update(1)

pbar.close()


pages inet:   0%|          | 0/5 [00:00<?, ?page/s]

[inet] page 1 -> https://inet.detik.com/indeks?page=1
  ditemukan 20 artikel
   + Miliaran Dolar Jadi 'Sampah'? Sisi Gelap di Balik Megaproyek AI
   + China Sukses Buat Prototipe EUV, Siap Produksi Chip 2nm
   + Apple Rilis iOS 26.3 Beta: Kini Bisa Transfer Data iPhone ke Android
   + Xiaomi 17 Ultra Debut 25 Desember: Bodinya Tipis, Kameranya Gahar
   + Ludes! Robot Anjing Wajah Elon Musk & Mark Zuckerberg Terjual Rp 1,5 M
   + Taksi Otonom Waymo Bikin Jalanan Lumpuh, Gegara Listrik Padam
   + Jadwal Games of the Future Mobile Legends Hari Ini: Onic Vs Aurora PH
   + Kisah Memalukan Tokyogurl, Bintang Esports Thailand Ketahuan Curang
   + Bukan Editan Photoshop! Foto Dijepret di Waktu Tepat Bikin Melotot
   + 10 Tempat Paling Berbahaya di Dunia yang Haram Didatangi
   + Ini Game PS5 & PS4 Terbaik 2025, Ghost of Yotei & Hollow Knight: Silksong
   + Komdigi Minta Netizen Jangan Terbelah Soal Bantuan Sumatera
   + Lolong: Buaya Terbesar di Dunia yang Pernah Diukur Hidup
   + Viral Laut I

pages inet:  20%|██        | 1/5 [00:48<03:14, 48.54s/page]

   + TelkomGroup Terjunkan 118 Relawan Dukung Pemulihan Pascabencana di Aceh
[inet] page 2 -> https://inet.detik.com/indeks?page=2
  ditemukan 20 artikel
   + Regulasi Makin Ketat, Infrastruktur Digital Harus Adaptif
   + Sisi Gelap di Balik Kecanggihan Robot Humanoid
   + Mesin Cuci Front Load 7KG Hanya Rp 4 Jutaan di Transmart Full Day Sale
   + Samsung Resmikan Exynos 2600, Siap Jadi Otaknya Galaxy S26
   + Riot Ungkap Celah di BIOS Motherboard Jadi Jalur Munculnya Cheat Game
   + Gokil, Kekayaan Elon Musk Tembus Rp 12.500 Triliun
   + Google Doodle Hari Ini: Bunga Cantik untuk Rayakan Hari Ibu 2025
   + Jadwal Games of the Future 2025 Mobile Legends: Onic Vs Team Falcons
   + Deretan Desain Bangku Taman Kreatif yang Nyeni
   + Elon Musk hingga Mark Zuckerberg 'Berubah Wujud' Jadi Robot Anjing
   + Steam Winter Sale Digelar, Ini Rekomendasi Game PC Diskon Hingga 90%
   + Murah Banget! Rekomendasi Game PC di Epic Games Store Diskon hingga 95%
   + Bibit Siklon Tropis 93S Aktif, BMKG 

pages inet:  40%|████      | 2/5 [01:39<02:29, 49.91s/page]

...

   + Inovasi Telkomsel Raih Outstanding Catalyst Award di Ajang TM Forum
[inet] page 4 -> https://inet.detik.com/indeks?page=4
  ditemukan 20 artikel
   + Krisis RAM Ancam Harga HP Naik di 2026, Ini Strategi Samsung Indonesia
   + Soundcore AeroClip dan C50i Resmi Hadir di RI, Harganya?
   + Viral Surat Edaran Harga Laptop Naik 2026, Asus dan Lenovo Buka Suara
   + Skandal Penjahat Seks Epstein: Foto Baru Ungkap Bill Gates & Sergey Brin
   + Naik Level! Robot Humanoid Jadi Penari Latar Penyanyi Top China
   + Awas! Nonton LK21 dan IndoXXI Bisa Bikin Rekening Ludes, Cek Link Aman
   + Hasil Esports SEA Games 2025: Indonesia Raih 1 Perak & 4 Perunggu
   + Kesiapan Jaringan Telekomunikasi Jelang Nataru
   + Adu Kemewahan: Deretan Foto Galaxy Z TriFold vs Fold7, Pilih Mana?
   + 15 Screenshot Chat Random Bikin Ngakak Nggak Ada Obatnya
   + EA Suntik Mati Real Racing 3 Usai 12 Tahun Mengaspal
   + Komdigi Desak Google Hapus 8 Aplikasi Data Mata Elang
   + Lele Raksasa Hampir 3 Meter Ditangk

pages inet:  80%|████████  | 4/5 [03:19<00:49, 49.96s/page]

   + Jelang Nataru, TelkomGroup Pastikan Layanan Berjalan Andal di Sumatera
[inet] page 5 -> https://inet.detik.com/indeks?page=5
  ditemukan 20 artikel
   + Coba Sebelum Beli, Strategi Dreame Edukasi Konsumen Soal Smart Cleaning
   + Samsung Galaxy Z TriFold Belum Masuk Indonesia, Ini Alasannya
   + Sosok Jay Ma, Selebgram Ngaku Kaya yang Diduga Anak Jack Ma
   + Instagram Resmi Batasi Pemakaian Hashtag: Maksimal Lima per Postingan
   + WFA 29-31 Desember 2025, Tanggung Nggak Sih?
   + Ragam Sad Food Netizen, Beli Online yang Datang Nggak Sesuai Harapan
   + Libur Nataru, Komdigi Siagakan 255 Posko Jaga Kualitas Sinyal
   + Bukan Buat Kaum Mendang-mending! Unboxing Samsung Galaxy Z TriFold
   + Nintendo Menang Gugatan Kontroler Wii, Kantongi Ganti Rugi Rp137 Miliar
   + Review Realme C85 Pro, HP Badak dengan Baterai Jumbo
   + Satelit Starlink 'Mati' dan Terjun Bebas ke Bumi
   + Benda Misterius Tabrak Bulan, Bikin Astronom Geger
   + Komdigi dan Opsel Siap Hadapi Lonjakan Trafik Libu

pages inet: 100%|██████████| 5/5 [04:12<00:00, 50.43s/page]

   + Telkomsel Ungkap Kendala Pemulihan Jaringan di Aceh, Banyak SUTET Roboh


In [ ]:
# Bentuk DataFrame dari seluruh hasil scraping (TANPA FILTER KOLOM)
df = pd.DataFrame(results)

# Buang baris yang benar-benar rusak (opsional tapi aman)
df.dropna(subset=["judul", "tanggal", "lead", "url"], how="all", inplace=True)

# Reset index
df.reset_index(drop=True, inplace=True)

print(f"Jumlah data total: {len(df)}")
display(df.head(10))


Jumlah data total: 88


,channel,url,judul,tanggal,tanggal_iso,lead
0,inet,https://inet.detik.com/business/d-8271552/mili...,Miliaran Dolar Jadi 'Sampah'? Sisi Gelap di Ba...,"Senin, 22 Des 2025 12:40 WIB",2025-12-22T12:40:00+07:00,Ada sebuah pertanyaan besar membayangi industr...
1,inet,https://inet.detik.com/consumer/d-8271225/chin...,"China Sukses Buat Prototipe EUV, Siap Produksi...","Senin, 22 Des 2025 07:15 WIB",2025-12-22T07:15:00+07:00,Industri semikonduktor China dilaporkan mencap...
2,inet,https://inet.detik.com/consumer/d-8271308/appl...,Apple Rilis iOS 26.3 Beta: Kini Bisa Transfer ...,"Senin, 22 Des 2025 09:15 WIB",2025-12-22T09:15:00+07:00,Apple belum lama ini merilis beta pertama iOS ...
3,inet,https://inet.detik.com/consumer/d-8272031/xiao...,Xiaomi 17 Ultra Debut 25 Desember: Bodinya Tip...,"Senin, 22 Des 2025 15:45 WIB",2025-12-22T15:45:00+07:00,Xiaomi resmi mengonfirmasi kehadiran smartphon...
4,inet,https://inet.detik.com/cyberlife/d-8271284/lud...,Ludes! Robot Anjing Wajah Elon Musk & Mark Zuc...,"Senin, 22 Des 2025 08:45 WIB",2025-12-22T08:45:00+07:00,Seniman digital Mike Winkelmann atau yang lebi...
5,inet,https://inet.detik.com/cyberlife/d-8271505/tak...,"Taksi Otonom Waymo Bikin Jalanan Lumpuh, Gegar...","Senin, 22 Des 2025 11:32 WIB",2025-12-22T11:32:00+07:00,Pemadaman listrik besar yang melanda San Franc...
6,inet,https://inet.detik.com/esport/d-8271456/jadwal...,Jadwal Games of the Future Mobile Legends Hari...,"Senin, 22 Des 2025 11:00 WIB",2025-12-22T11:00:00+07:00,Setelah menang dari Team Falcons di Quarterfin...
7,inet,https://inet.detik.com/esport/d-8271907/kisah-...,"Kisah Memalukan Tokyogurl, Bintang Esports Tha...","Senin, 22 Des 2025 14:30 WIB",2025-12-22T14:30:00+07:00,"Warasin 'Tokyogurl' Naraphat, yang pernah dipu..."
8,inet,https://inet.detik.com/fotoinet/d-8271259/buka...,Bukan Editan Photoshop! Foto Dijepret di Waktu...,"Senin, 22 Des 2025 08:15 WIB",2025-12-22T08:15:00+07:00,Pool - detikInet
9,inet,https://inet.detik.com/fotoinet/d-8271403/10-t...,10 Tempat Paling Berbahaya di Dunia yang Haram...,"Senin, 22 Des 2025 10:15 WIB",2025-12-22T10:15:00+07:00,Danau Karachay yang terletak di wilayah Chelya...


In [ ]:
csv_path = "/content/inet_detik_raw_results.csv"
json_path = "/content/inet_detik_raw_results.json"

# Simpan CSV (Excel-friendly)
df.to_csv(
    csv_path,
    index=False,
    encoding="utf-8-sig"
)

# Simpan JSON (NLP / Python friendly)
df.to_json(
    json_path,
    orient="records",
    force_ascii=False,
    indent=2
)

print("File berhasil disimpan:")
print(f"CSV  -> {csv_path}")
print(f"JSON -> {json_path}")






File berhasil disimpan:
CSV  -> /content/inet_detik_raw_results.csv
JSON -> /content/inet_detik_raw_results.json
